Задание 1.

1.1. Разобраться в функциях sklearn GridSearch, RandomSearch для дерева решений и логистической регрессии.

1.2. Сравнить результаты для обоих моделей. 

Проблемы с установкой graphviz.

Возможное решение: https://overcoder.net/q/463149/python-%D0%BF%D1%80%D0%BE%D0%B1%D0%BB%D0%B5%D0%BC%D1%8B-%D1%81-%D1%83%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%BA%D0%BE%D0%B9-pygraphviz-%D0%B8-graphviz

Задание 2.

2.1. Взять исходный датасет Titanic и попробовать не выбрасывать строковые переменные, а закадировать их. 
Вам могут помочь LabelEncoder, OneHotEncoder и др, которые можно увидеть в соответствующих библиотеках sklearn. 
Сравнить полученные результаты со случаем, когда строковые переменные не использовались. 
И там и там используйте модель с гиперпараметрами, найденными в первом пункте.

Полезные ссылки:
    
1) Отличия LabelEncoder от OneHotEncoder: https://habr.com/ru/post/456294/

In [7]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [13]:
#Запуск случайных чисел
seed = 42
np.random.seed(seed)

# Чтение файла с обучающейся выборкой
data = pd.read_csv('../../datasets/in/titanic/train.csv')

# Присвоение переменным значений с количеством спасшихся и с остальными данными
y = data['Survived']
x = data.drop(['Survived'], axis=1)

# Функция по переводу строкового признака Пол в числовой
def binarize(x):
    if x == 'female':
        return 1
    else:
        return 0

# Перевод строкового признака Пол в числовой
x['Sex'] = x['Sex'].apply(lambda d: binarize(d))

# Добавление столбца с признаком удаленого события
x['Age_unk'] = x['Age'].apply(lambda d: np.isnan(d))

# Замена нулевых значений медианой
x['Age'].fillna(x['Age'].median(), inplace=True)

# Метод позволяющей поделить данные на 20 и 80%
X_train, X_test, y_train, y_test = train_test_split(x,
                                                   y, test_size=0.2,
                                                   random_state=seed)

In [9]:
# Присвоение значений признаков для обучающей и тестовой выборке переменным
X_train_drop = X_train
X_test_drop = X_test

X_train_encoder = X_train
X_test_encoder = X_test

In [10]:
# Удаление лишних (не числовых столбцов)
drop = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
X_train_drop.drop(columns=drop, inplace=True)
X_train_drop.drop(columns=drop, inplace=True)

Задание 3.

3.1. Посмотреть метрики вида (в библиотеке sklearn) Precision, Accuracy, Recall и F1 score. 

3.2. Понять где они наиболее приемлемы для использования. 

3.3. Сравнить результаты модели в различных метриках. Модель наилучшая после двух пунктов.

Ссылки:

1) Теория по метрикам кода (видеолекция на 8,5 мин.): https://courses.openedu.ru/courses/course-v1:mipt+DATA_AN+spring_2020/courseware/309ba8b3850e4278a9522d2c4e0e5135/4e2d182dbda54c3595a1632862257f4b/;
